In [5]:
import pandas as pd
import json

In [6]:
json1_file = open('./CleanedData/Council_to_Code.json')
json1_str = json1_file.read()
code_dict = json.loads(json1_str)

In [7]:
df_tercile = pd.read_csv("./CleanedData/Income_Tercile_Map.csv")
df_tercile = df_tercile.tail(-1)
df_tercile.rename(columns = {"Year": "Council Code"}, inplace=True)
df_tercile

,Council Code,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,Aberdeen City,High,High,High,High,High,High,High,High,High,High
2,Aberdeenshire,Low,Low,Mid,Mid,Mid,Mid,Mid,Mid,Mid,Mid
3,Adur,Mid,Mid,Mid,Low,Low,Low,Low,Mid,Low,Low
4,Amber Valley,High,High,Mid,High,Mid,Low,Low,Low,Mid,Mid
5,Angus,Low,Low,Low,Low,Low,Low,Mid,Low,Mid,Low
...,...,...,...,...,...,...,...,...,...,...,...
359,Wychavon,Mid,Mid,Mid,Mid,Low,Mid,Mid,High,High,Mid
360,Wyre,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
361,Wyre Forest,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
362,York UA,Mid,Mid,Low,Mid,Mid,Mid,High,Mid,High,Mid


In [8]:
tercile = dict(zip(df_tercile["Council Code"], df_tercile["2016"]))


In [ ]:
for year in range (2016, 2025):
    if (year < 2022):
        df= pd.read_csv(f"./Temp/Migration_{year}.csv")
    else:
        df= pd.read_excel(f"./Data/Migration/Migration_{year}.xlsx")
    tercile = dict(zip(df_tercile["Council Code"], df_tercile[f"{year}"]))
    df.rename(columns={'outla': "Origin"}, inplace=True)
    df.rename(columns={'inla': "Destination"}, inplace=True)
    if (year < 2022):
        df.drop(columns=["Year", "sex"], axis=1, inplace=True)
    else:
        df.drop(columns=["year", "sex"], axis=1, inplace=True)
    df.replace({"Destination": code_dict}, inplace=True)
    df.replace({"Origin": code_dict}, inplace=True)
    df["Origin"] = df["Origin"].str.strip()
    df["Destination"] = df["Destination"].str.strip()
    df.replace({"Origin": tercile}, inplace=True)
    df["0-16"] = df.loc[:, "Age_0":"Age_16"].sum(axis=1)
    df["17-34"] = df.loc[:, "Age_17":"Age_34"].sum(axis=1)
    df["35-54"] = df.loc[:, "Age_35":"Age_54"].sum(axis=1)
    df["55-66"] = df.loc[:, "Age_55":"Age_66"].sum(axis=1)
    df["67+"] = df.loc[:, "Age_67":].sum(axis=1)
    df.drop(columns=[c for c in df.columns if c not in ["Origin", "Destination", "0-16", "17-34", "35-54", "55-66", "67+"]], inplace=True)
    df = df[["Destination", "Origin", '0-16', "17-34", "35-54", "55-66", "67+"]] 
    df.sort_values(by="Destination", inplace=True)
    df = df[df["Origin"].isin(["Mid", "Low", "High"])]
    df = df.groupby(["Origin", "Destination"], as_index=False).sum()
    df.sort_values(by="Destination", inplace=True)
    df.to_csv(f"./CleanedData/Migration_Cleaned_{year}.csv", index=True)  

In [10]:
age_cols = ['0-16', '17-34', '35-54', '55-66', '67+']

all_years = []

for year in range(2016, 2025):
    df = pd.read_csv(f"./Temp/Migration_Cleaned_{year}.csv")
    income_sums = (df.groupby(['Destination', 'Origin'])[age_cols].sum().sum(axis=1).unstack('Origin').add_prefix('income_'))
    age_sums = (df.groupby('Destination')[age_cols].sum())
    result = income_sums.join(age_sums)
    combo = (df.groupby(['Destination', 'Origin'])[age_cols].sum().unstack('Origin'))
    combo.columns = [f'{origin}_{age}' for age, origin in combo.columns]
    final_df = result.join(combo)
    final_df.index = final_df.index.map(lambda x: f"{x}_{year}")
    all_years.append(final_df)
full_df = pd.concat(all_years)


In [11]:
full_df.to_csv(f"./CleanedData/Income_Inequality_Regression_Ready.csv", index=True)   
